# **Tech Challenge - Fase 1**

---

## **O Problema**
Você é um profissional encarregado de desenvolver um modelo preditivo de regressão para prever o valor dos custos médicos individuais cobrados pelo seguro de saúde.

A base de dados para este desafio é a **seguro_saude.csv** na pasta **Base de Dados**.

---

## **Tarefas**

### **Exploração de dados:**
- Carregue a base de dados e explore suas características.
- Analise estatísticas descritivas e visualize distribuições relevantes.

### **Pré-processamento de dados:**
- Realize a limpeza dos dados, tratando valores ausentes (se necessário).
- Converta variáveis categóricas em formatos adequados para modelagem.

### **Modelagem:**
- Crie um modelo preditivo de regressão utilizando uma técnica à sua escolha (por exemplo, Regressão Linear, Árvore de Decisão, etc...).
- Divida o conjunto de dados em conjuntos de treinamento e teste.

### **Treinamento e avaliação do modelo:**
- Treine o modelo com conjunto de treinamento.

### **Validação estatística:**
- Utilize métricas estatísticas para validar a eficácia do modelo (p-value, intervalos de confiança).

---